In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [4]:
KEY=os.getenv("OPENAI_API_KEY")

In [5]:
print(KEY)

aa-Fms2VpR5ZCcltiS6sGisGgD4KJGBFAYFgZRDjL4cn8ysArye


In [6]:
#KEY="aa-NRk18JU4VWjrrmmW5UlCDSG5eHhEy70XT0LxoblcBpGf1GIn"

In [7]:
llm = ChatOpenAI(model="gpt-3.5-turbo", base_url="https://api.avalai.ir/v1", api_key=KEY, temperature=0.5)

#llm.invoke("")

C:\Users\Mohammad\AppData\Local\Temp\ipykernel_4632\3675271060.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", base_url="https://api.avalai.ir/v1", api_key=KEY, temperature=0.5)


In [8]:
# from langchain_openai import OpenAI  # pip install -U langchain_openai

# llm = OpenAI(
#     model="gpt-3.5-turbo-instruct",
#     base_url="https://api.avalai.ir/v1", 
#     api_key="KEY"
# )

In [9]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000219FFA83FD0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000219FFACA110>, temperature=0.5, model_kwargs={}, openai_api_key='aa-Fms2VpR5ZCcltiS6sGisGgD4KJGBFAYFgZRDjL4cn8ysArye', openai_api_base='https://api.avalai.ir/v1', openai_proxy='')

In [10]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [11]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [12]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [13]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [14]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Mohammad\AppData\Local\Temp\ipykernel_4632\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [17]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [18]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [19]:
file_path="G:/AbzuAI/classLLM30h/MCQ_proj/data.txt"

In [20]:
file_path

'G:/AbzuAI/classLLM30h/MCQ_proj/data.txt'

In [21]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [22]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [23]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [24]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [25]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Mohammad\AppData\Local\Temp\ipykernel_4632\1669711107.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [27]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt (input) Tokens:{cb.prompt_tokens}")
print(f"Completion (output) Tokens:{cb.completion_tokens}")
print(f"Total Cost (dollar):{cb.total_cost}")

Total Tokens:1779
Prompt (input) Tokens:1192
Completion (output) Tokens:587
Total Cost (dollar):0.002962


In [28]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [33]:
quiz=response.get("quiz")

In [ ]:
json.loads(quiz)


{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Photosynthesis',
   'c': 'Mitosis',
   'd': 'Osmosis'},
  'correct': 'a'},
 '3': {'mcq': 'What is the basic unit of life that processes hereditary information?',
  'options': {'a': 'Atoms', 'b': 'Molecules', 'c': 'Cells', 'd': 'Tissues'},
  'correct': 'c'},
 '4': {'mcq': 'Which process allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Photosynthesis',
   'b': 'Respiration',
   'c': 'Fermentation',
   'd': 'Oxidation'},
  'correct': 'b'},
 '5': {'mcq': 'What is the study of the anatomy and physiology of plants and animals called?',
  'options': {'a': 'Zoology',
   'b': 'Botany',
   'c': 'Anatomy',
   'd': 'Physiology'},
  'correct': 'c'}}

In [35]:
quiz=json.loads(quiz)

In [36]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [37]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'Which theme explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Photosynthesis | c: Mitosis | d: Osmosis',
  'Correct': 'a'},
 {'MCQ': 'What is the basic unit of life that processes hereditary information?',
  'Choices': 'a: Atoms | b: Molecules | c: Cells | d: Tissues',
  'Correct': 'c'},
 {'MCQ': 'Which process allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Photosynthesis | b: Respiration | c: Fermentation | d: Oxidation',
  'Correct': 'b'},
 {'MCQ': 'What is the study of the anatomy and physiology of plants and animals called?',
  'Choices': 'a: Zoology | b: Botany | c: Anatomy | d: Physiology',
  'Correct': 'c'}]

In [38]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,What is the scientific study of life called?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
1,Which theme explains the unity and diversity o...,a: Evolution | b: Photosynthesis | c: Mitosis ...,a
2,What is the basic unit of life that processes ...,a: Atoms | b: Molecules | c: Cells | d: Tissues,c
3,"Which process allows organisms to move, grow, ...",a: Photosynthesis | b: Respiration | c: Fermen...,b
4,What is the study of the anatomy and physiolog...,a: Zoology | b: Botany | c: Anatomy | d: Physi...,c


In [39]:
quiz=pd.DataFrame(quiz_table_data)

In [40]:
quiz.to_csv("machinelearning.csv",index=False)

In [1]:
from datetime import datetime
datetime.now()

datetime.datetime(2024, 12, 6, 15, 6, 28, 718354)

In [2]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'12_06_2024_15_06_41'